In [1]:
#获取索引

import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json

gheaders = {
    'Referer': 'https://ieeexplore.ieee.org/xpl/issues?punumber=90&isnumber=9169972',#网站反爬
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
    }
res = requests.get('https://ieeexplore.ieee.org/rest/publication/90/regular-issues',headers=gheaders)
data = res.text

volume = re.findall('"volume":"(.*?)","issue":".*?".*?,"issueNumber":.*?,.*?"year":".*?","month":".*?"',data)
issue = re.findall('"volume":".*?","issue":"(.*?)".*?,"issueNumber":.*?,.*?"year":".*?","month":".*?"',data)
issuenumber = re.findall('"volume":".*?","issue":".*?".*?,"issueNumber":(.*?),.*?.*?"year":".*?","month":".*?"',data)
year = re.findall('"volume":".*?","issue":".*?".*?,"issueNumber":.*?,.*?"year":"(.*?)","month":".*?"',data)
month = re.findall('"volume":".*?","issue":".*?".*?,"issueNumber":.*?,.*?"year":".*?","month":"(.*?)"',data)

data_all = {
    'volume':np.array(volume),
    'year':np.array(year),
    'month':np.array(month),
    'issue':np.array(issue),
    'issuenumber':np.array(issuenumber)
}
da = pd.DataFrame(data_all)
da

,volume,year,month,issue,issuenumber
0,28,2020,Aug.,4,9169972
1,28,2020,June,3,9119211
2,28,2020,April,2,9069346
3,28,2020,Feb.,1,8999818
4,27,2019,Dec.,6,8935550
...,...,...,...,...,...
161,1,1993,Oct.,5,6439
162,1,1993,Aug.,4,6438
163,1,1993,June,3,6050
164,1,1993,April,2,5815


In [2]:
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json


def find_year(da):
    return da.index.str.startswith("2020")==1

#存储为excel
writer = pd.ExcelWriter(r'D:/jupyter_save/data.xlsx')
da['date'] = da['year'].map(str) + "-" + da['month']
da.set_index('date', inplace = True)
#str.replace('*','*')
da.loc[:,'issuenumber'] = da['issuenumber'].astype('int64')
da.to_excel(writer,sheet_name = 'IEEE Communications Magazine',index=True)
writer.save()
#数据爬取处理预备
file = pd.ExcelFile(r'./data.xlsx')
handle = pd.read_excel('./data.xlsx')
issuenum = handle['issuenumber']
datestr = handle['date'] 
#test
da.loc[find_year,:]


,volume,year,month,issue,issuenumber
date,,,,,
2020-Aug.,28,2020,Aug.,4,9169972
2020-June,28,2020,June,3,9119211
2020-April,28,2020,April,2,9069346
2020-Feb.,28,2020,Feb.,1,8999818


In [3]:
issuenum

0      9169972
1      9119211
2      9069346
3      8999818
4      8935550
        ...   
161       6439
162       6438
163       6050
164       5815
165       5814
Name: issuenumber, Length: 166, dtype: int64

In [ ]:
from lxml import etree
import json
import time
#id储存在toc数据包
length=len(issuenum)

alf = open(r'save.txt', 'w')

def get_paperinfo(issuenumber):
    jsonerror_flag = 0
    name = pd.Series([0])
    for x in range(122,124):#一次传太多容易报错，改这里，有多少个运行上面第一栏代码可以得到，筛选最新的
        time.sleep(1)
        print(x)
        y = issuenumber.loc[x]
        y = str(y)
        gheaders = {
            'Host': 'ieeexplore.ieee.org',
            'Referer': 'https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber='+y+'&punumber=90',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
            }
        toc_url = 'https://ieeexplore.ieee.org/rest/search/pub/90/issue/'+y+'/toc'
        for page in range(0,3):
            if page==1:
                payload = '{"isnumber":"'+y+'","punumber":"90","sortType":"vol-only-seq","pageNumber":"2"}'
            elif page==2:
                payload = '{"isnumber":"'+y+'","punumber":"90","sortType":"vol-only-seq","pageNumber":"3"}'
            else:
                payload = '{"isnumber":"'+y+'","punumber":"90","sortType":"vol-only-seq"}'     
            toc_res = requests.post(toc_url, headers = gheaders,data = payload)
            try:
                toc_dic = json.loads(toc_res.text)
                try:
                    articles = toc_dic['records']
                except KeyError:
                    print('不存在第'+str(page+1)+'页')
                    break
                else:
                    for article in articles:
                        flag = article['ephemera']
                        if flag == False:
                            #title = article['highlightedTitle']
                            paperid = article['articleNumber']
                            alf.write(paperid+'\n')
                            print(paperid)
            except json.JSONDecodeError:
                print('JSON error,try again-------issuenumber=%s', y )
                toc_url = 'https://ieeexplore.ieee.org/rest/search/pub/35/issue/'+y+'/toc'
                payload = '{"isnumber":"'+y+'","punumber":"90","sortType":"vol-only-seq"}'
                toc_res = requests.post(toc_url, headers = gheaders,data = payload)
                try:
                    toc_dic = json.loads(toc_res.text)
                    try:
                        articles = toc_dic['records']
                    except KeyError:
                        print('KEY_ERROR,filenum=',x)
                        break
                    else:
                        for article in articles:
                            flag = article['ephemera']
                            if flag == False:
                                #title = article['highlightedTitle']
                                paperid = article['articleNumber']
                                alf.write(paperid+'\n')
                                print(paperid)
                except json.JSONDecodeError:
                    name.loc[jsonerror_flag] = y
                    jsonerror_flag = jsonerror_flag + 1
                    print('Error  again,Pass\n')
                    continue
                else:
                    continue
            else:
                continue
    alf.close()
get_paperinfo(issuenum)
        


    



In [9]:
#对得到的id整合
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json


id_data = pd.read_csv(r'./save.txt',header=None)
id_data.rename(columns={0:'paper-ID'}, inplace = True)
writer_paperid = pd.ExcelWriter(r'D:/jupyter_save/paperid.xlsx')
id_data.to_excel(writer_paperid,sheet_name = 'paper_idsave',index=True)
writer_paperid.save()
print('Finish')

Finish


In [10]:
#对所有文章进行信息获取
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json

id_handle = pd.read_excel(r'./paperid.xlsx')
id_data = id_handle['paper-ID']
len(id_data)

3440

In [5]:
#对所有文章进行信息获取
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json

id_handle = pd.read_excel(r'./paperid.xlsx')
id_data = id_handle['paper-ID']
#id_data
info_creat = pd.DataFrame({'Year':[],
                          'Month':[],
                          'Day':[],
                          'Name':[],
                          'Keyword_0':[],
                          'Keyword_1':[],
                          'Keyword_2':[],
                          'Keyword_3':[],
                          'Keyword_4':[]                                                       
                         })
info_save = info_creat
def get_info(id_data, info_save,info_creat):
    finish_flag = False
    max_index = 3440
    median_info = info_creat
    print('一组20篇文章，组数如下')
    for x in range(160,180):
        times = x+1
        print(times)
        if finish_flag == False:
            for y in range(0,20):
                flag = x*20 + y
                if flag < max_index:
                    id_test = id_data.loc[flag]
                    id_test = str(id_test)
                    gheaders = {
                    'Host': 'ieeexplore.ieee.org',
                    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
                    'Referer': 'https://ieeexplore.ieee.org/document/'+id_test+'/keywords',
                    }
                    metadata_url='https://ieeexplore.ieee.org/document/'+id_test+'/keywords'
                    #try看情况是否要添加
                    meta_res = requests.get(metadata_url,headers=gheaders)
                    info_data = meta_res.text
                    #global.document.metadata.*?"Keywords","kwd":(.*?)}],"isGetArticle".*?"journalDisplayDateOfPublication":.*?,"isPromo"
                    keywords = re.findall('"IEEE Keywords","kwd":(.*?)]}',info_data)
                    pub=re.findall('"journalDisplayDateOfPublication":(.*?),',info_data)
                    paper=re.findall('<title>(.*?) - IEEE',info_data)
                    keyword=''.join(keywords)
                    paper_name=''.join(paper)
                    pub=''.join(pub)
                    pub_date=pub.replace('"','')
                    keyword=keyword.replace('"','')
                    keyword=keyword.replace('[','')
                    keyword=keyword.replace(']','')
                    keyword_all=keyword.split(',')
                    pub_date=pub_date.split(' ')
                    if len(pub_date) == 3:
                        table_creat = pd.DataFrame({'Year':[pub_date[2]],
                                                    'Month':[pub_date[1]],
                                                    'Day':[pub_date[0]],
                                                    'Name':[paper_name],                                                   
                                                   })
                        for i in range(0,len(keyword_all)):
                            name_i = str(i)
                            table_creat.loc[:,"Keyword_"+name_i]=[keyword_all[i]]
                            #备忘：这里是当前文章的一行信息
                        median_info = pd.concat([median_info,table_creat],ignore_index = True)
                        #备忘：median_info循环20遍记录数据
                    else:
                        pub_date.insert(0,'00')
                        table_creat = pd.DataFrame({'Year':[pub_date[2]],
                                                    'Month':[pub_date[1]],
                                                    'Day':[pub_date[0]],
                                                    'Name':[paper_name],                                                   
                                                   })
                        for i in range(0,len(keyword_all)):
                            name_i = str(i)
                            table_creat.loc[:,"Keyword_"+name_i]=[keyword_all[i]]
                            #备忘：这里是当前文章的一行信息
                        median_info = pd.concat([median_info,table_creat],ignore_index = True)
                        #备忘：median_info循环20遍记录数据
                else:
                    finish_flag = True
                    break
            info_save = pd.concat([info_save,median_info],ignore_index = True)
            median_info = info_creat
            print('第'+str(times)+'组完成')
        else:
            info_save = pd.concat([info_save,median_info],ignore_index = True)
            median_info = info_creat
            print('第'+str(times)+'组完成,无剩余内容')
            break
    return info_save 
info_data = get_info(id_data, info_save,info_creat)
paper_info = pd.ExcelWriter(r'D:/jupyter_save/paper_info.xlsx')
info_data.to_excel(paper_info,sheet_name = 'IEEE Communications Magazine',index=False)
paper_info.save()
runned_flag = True
print('Finish')

一组20篇文章，组数如下
161
第161组完成
162
第162组完成
163
第163组完成
164
第164组完成
165
第165组完成
166
第166组完成
167
第167组完成
168
第168组完成
169
第169组完成
170
第170组完成
171
第171组完成
172
第172组完成
173
第173组完成
174
第174组完成,无剩余内容
Finish


In [15]:
runned_flag

False

In [ ]:
info_data

In [6]:
#分批录入

import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json
if runned_flag ==True:
    merged_writer = pd.ExcelWriter(r'D:/jupyter_save/total_info.xlsx')
    merge_info_now = pd.read_excel(r'./paper_info.xlsx')
    merge_info_pro = pd.read_excel(r'./total_info.xlsx')
    merge_allinfo = pd.concat([merge_info_pro,merge_info_now],ignore_index = True)
    merge_allinfo.to_excel(merged_writer,sheet_name = 'IEEE Transactions on Networking',index=False)
    merged_writer.save()
    runned_flag =False
    print('Finish')
else:
    print('无需重复录入')

Finish


In [13]:
# #建立此期刊对应sheet且会初始化！！！
# import requests
# import re
# import pandas as pd
# import numpy as np
# import xlwt
# from lxml import etree
# import json

# This = pd.DataFrame({'Year':[],
#                           'Month':[],
#                           'Day':[],
#                           'Name':[],
#                           'Keyword_0':[],
#                           'Keyword_1':[],
#                           'Keyword_2':[],
#                           'Keyword_3':[],
#                           'Keyword_4':[]                                                       
#                          })
# writer = pd.ExcelWriter(r'D:/jupyter_save/total_info.xlsx')
# This.to_excel(writer,sheet_name = 'IEEE Transactions on Networking',index=False)
# writer.save()
# print('Finish')


Finish


In [123]:
#     #测试用

#     import requests
#     import re
#     import pandas as pd
#     import numpy as np
#     import xlwt
#     from lxml import etree
#     import json
#     merged_writer = pd.ExcelWriter(r'D:/jupyter_save/total_info.xlsx')
#     merge_info_now = pd.read_excel(r'./paper_info.xlsx')
#     merge_info_pro = pd.read_excel(r'./total_info.xlsx')
#     merge_allinfo = pd.concat([merge_info_pro,merge_info_now],ignore_index = True)
#     merge_allinfo.to_excel(merged_writer,sheet_name = 'IEEE Communications Magazine',index=False)
#     merged_writer.save()
    
#     print('Finish')
#     merge_info_pro

Finish


,Year,Month,Day,Name,Keyword_0,Keyword_1,Keyword_2,Keyword_3,Keyword_4,Keyword_5,Keyword_6,Keyword_7
0,2020,August,7,"Wake-up Radio-Based 5G Mobile Access: Methods,...",Power demand,Synchronization,Delays,5G mobile communication,Energy consumption,Batteries,3GPP,NaN
1,2020,August,7,Advanced Dynamic Spectrum 5G Mobile Networks E...,Sensors,Resource management,Europe,5G mobile communication,Synthetic aperture sonar,Broadband communication,3GPP,NaN
2,2020,August,7,A Hierarchical and Modular Radio Resource Mana...,5G mobile communication,Resource management,Complexity theory,Wireless communication,Heterogeneous networks,Licenses,Long Term Evolution,NaN
3,2020,August,7,BAGKD: A Batch Authentication and Group Key Di...,Authentication,Protocols,Vehicular ad hoc networks,Public key,Road transportation,NaN,NaN,NaN
4,2020,August,7,Automating Vertical Services Deployments over ...,5G mobile communication,Industries,Network slicing,Cloud computing,Systems architecture,Software defined networking,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
75,2020,June,9,A Hybrid Positioning System for Location-Based...,Bluetooth,Protocols,Servers,Wireless fidelity,Zigbee,Robustness,Position measurement,NaN
76,2020,April,20,Toward Operator-to-Waveform 5G Radio Access Ne...,5G mobile communication,IP networks,Radio access networks,Complexity theory,Quality of experience,Network slicing,NaN,NaN
77,2020,April,20,LightEdge: Mapping the Evolution of Multi-Acce...,5G mobile communication,IP networks,Edge computing,Internet,Computer architecture,3GPP,NaN,NaN
78,2020,April,20,End-to-End Network Slicing for Flash Crowds,Network slicing,Wireless communication,Virtualization,5G mobile communication,Software engineering,Cloud computing,NaN,NaN


In [24]:
#统一不同时期的格式并存储
import requests
import re
import pandas as pd
import numpy as np
import xlwt
from lxml import etree
import json
import openpyxl


def add_sheet(data, excel_writer, sheet_name):
    book = openpyxl.load_workbook(excel_writer.path)
    excel_writer.book = book
    data.to_excel(excel_writer=excel_writer, sheet_name=sheet_name, index=True, header=True)
    excel_writer.close()
    
    
    
writer = pd.ExcelWriter(r'D:/jupyter_save/Final_info.xlsx', engine='openpyxl')
info = pd.read_excel(r'./total_info.xlsx')
info['Date'] = info['Month'].map(str)  + info['Day'].map(str)
info_fix = info['Date'].str.replace("[0123456789.]","").str.replace('nan','')
info_fix = info_fix.str.replace('Jan','01').str.replace('Feb','02').str.replace('Mar','03').str.replace('Apr','04').str.replace('May','05').str.replace('Nov','11')
info_fix = info_fix.str.replace('Jun','06').str.replace('Jul','07').str.replace('Aug','08').str.replace('Sep','09').str.replace('Oct','10').str.replace('Dec','12')
info_fix = info_fix.str.replace("[abcehilmorstuy]","")
info['Date'] = info_fix
info['Date'] = info['Year'].map(str)  +'-'+ info['Date'].map(str)
# info['Date'] = info['Date'].str.replace('\n','')
del info['Month']
del info['Day']
del info['Year']
info.set_index('Date',inplace = True)

add_sheet(info, writer, 'IEEE Transactions on Networking')
# info.to_excel(writer,sheet_name = 'IEEE Transactions on Networking',index=True)
# writer.save()
print('finish')


finish
